# 마법사 상어와 파이어스톰

[baekjoon의 '마법사 상어와 파이어스톰' 링크](https://www.acmicpc.net/problem/20058)

### 문제  
마법사 상어는 파이어볼과 토네이도를 조합해 파이어스톰을 시전할 수 있다. 오늘은 파이어스톰을 크기가 $2^N$ × $2^N$인 격자로 나누어진 얼음판에서 연습하려고 한다. 위치 (r, c)는 격자의 r행 c열을 의미하고, A[r][c]는 (r, c)에 있는 얼음의 양을 의미한다. A[r][c]가 0인 경우 얼음이 없는 것이다.  
  
파이어스톰을 시전하려면 시전할 때마다 단계 L을 결정해야 한다. 파이어스톰은 먼저 격자를 $2^L$ × $2^L$ 크기의 부분 격자로 나눈다. 그 후, 모든 부분 격자를 시계 방향으로 90도 회전시킨다. 이후 얼음이 있는 칸 3개 또는 그 이상과 인접해있지 않은 칸은 얼음의 양이 1 줄어든다. (r, c)와 인접한 칸은 (r-1, c), (r+1, c), (r, c-1), (r, c+1)이다. 아래 그림의 칸에 적힌 정수는 칸을 구분하기 위해 적은 정수이다.  
  
||||
|:---:|:---:|:---:|
|![](https://upload.acmicpc.net/68137f5d-fdbd-48c6-92f0-0a74ee53b0c2/-/preview/)|![](https://upload.acmicpc.net/4216e4de-a9f7-4bf0-9385-e20c583c1228/-/preview/)|![](https://upload.acmicpc.net/a58a4219-afc7-4f77-a194-a5495882eeb4/-/preview/)|
|마법을 시전하기 전|	L = 1|	L = 2|  
  
마법사 상어는 파이어스톰을 총 Q번 시전하려고 한다. 모든 파이어스톰을 시전한 후, 다음 2가지를 구해보자.  
  
1. 남아있는 얼음 A[r][c]의 합  
2. 남아있는 얼음 중 가장 큰 덩어리가 차지하는 칸의 개수  
  
얼음이 있는 칸이 얼음이 있는 칸과 인접해 있으면, 두 칸을 연결되어 있다고 한다. 덩어리는 연결된 칸의 집합이다.  
  
### 입력  
첫째 줄에 N과 Q가 주어진다. 둘째 줄부터 $2^N$개의 줄에는 격자의 각 칸에 있는 얼음의 양이 주어진다. r번째 줄에서 c번째 주어지는 정수는 A[r][c] 이다.  
  
마지막 줄에는 마법사 상어가 시전한 단계 $L_1, L_2, ..., L_Q$가 순서대로 주어진다.  
  
### 출력  
첫째 줄에 남아있는 얼음 A[r][c]의 합을 출력하고, 둘째 줄에 가장 큰 덩어리가 차지하는 칸의 개수를 출력한다. 단, 덩어리가 없으면 0을 출력한다.  
  
### 제한  
- 2 ≤ N ≤ 6
- 1 ≤ Q ≤ 1,000
- 0 ≤ A[r][c] ≤ 100
- 0 ≤ $L_i$ ≤ N  

In [21]:
import numpy as np
from queue import Queue
from pprint import pprint

def rotate_ice(N, ice, y, x, size): # 나누어진 격자를 90도 회전시키는 함수
    init = [] 
    for i in range(size):
        init.append(ice[y+i][x:x+size])
    # 회전시켜야할 격자 행렬
    rotated = np.rot90(init, k=-1).tolist() # 해당 격자를 90도 회전시킨 행렬
    for add_y in range(size):
        for add_x in range(size):
            ice[y+add_y][x+add_x] = rotated[add_y][add_x]
    # 그리고 ice에 회전시킨 행렬값으로 갱신
    return

def check_ice(N, ice, checked, init_y, init_x): # 모든 파이어스톰 후 얼음덩어리의 상태 체크하는 함수
    total_ice = 0 #  얼음덩어리에서의 총 얼음 개수
    total_num = 0 # 얼음덩어리의 크기
    direction = [(1, 0), (-1, 0), (0, 1), (0, -1)] # 방향
    q = Queue() # 큐
    q.put((init_y, init_x)) # 현재 좌표를 큐에 push
    while not q.empty():
        y, x = q.get() # (y, x) 좌표를 큐에서 pop
        for add_y, add_x in direction: # 인접한 네 방향에 대해서
            if 0 <= add_y+y < pow(2, N) and 0 <= add_x+x < pow(2, N): # 보드 내의 범위일 때
                if ice[add_y+y][add_x+x] == 0: # 얼음이 없는 좌표라면 탐색하지 않음
                    continue
                if not checked[add_y+y][add_x+x]: # 얼음이 있음에도 아직 탐색하지 않은 위치라면
                    checked[add_y+y][add_x+x] = True
                    q.put((add_y+y, add_x+x)) # 해당 위치를 큐에 push한다.
        total_ice += ice[y][x] # 현재 탐색 중인 좌표 위의 얼음 개수를 합한다
        total_num += 1 # 얼음덩어리 크기를 1 증가시킨다.
    return total_ice, total_num # 총 얼음 개수, 얼음덩어리 크기 반환
        

def simulation(N, Q, ice, L): # 파이어스톰 시뮬레이션 진행
    max_val = 0 # 얼음 덩어리 중에 가장 큰 얼음 덩어리 사이즈
    sum_val = 0 # 보드 위의 총 얼음 개수
    direction = [(1, 0), (-1, 0), (0, 1), (0, -1)]
    for l in L: # 모든 파이어스톰에 대해서
        Q -= 1
        if l > 0: # 1칸보다 더 큰 격자로 나누어질 경우
            size = pow(2, l)
            for y in range(0, pow(2, N), size):
                for x in range(0, pow(2, N), size):
                    rotate_ice(N, ice, y, x, size)
            # 나누어진 모든 격자에 대해서 각각 90도 회전시킨다.
        
        melt_list = [] # 얼음이 녹아야되는 좌표 리스트
        for y in range(pow(2, N)):
            for x in range(pow(2, N)): # 보드 위의 모든 좌표에 대해서
                if ice[y][x] == 0: # 얼음이 없는 곳은 제외하고
                    continue
                count = 4
                for add_y, add_x in direction: # 탐색 중인 좌표의 인접한 네 방향에 대해서
                    if count < 3:
                        break
                    if 0 <= add_y+y < pow(2, N) and 0 <= add_x+x < pow(2, N) and ice[y+add_y][x+add_x] > 0:
                        continue
                    count -= 1 # 인접한 방향으로 이동 시 보드를 벗어나거나, 인접한 방향에 얼음이 없을 경우 체크
                if count < 3: # 인접한 네 방향 중에 얼음이 존재하는 곳이 3곳보다 작다면
                    melt_list.append((y, x)) # 얼음이 녹아야되는 좌표에 추가
        for y, x in melt_list:
            ice[y][x] -= 1
        # 위에서 추가한 좌표에서 얼음을 1씩 녹인다
                    

    checked = [[False for _ in range(pow(2, N))] for _ in range(pow(2, N))]
    for y in range(pow(2, N)):
        for x in range(pow(2, N)): # 모든 보드 위의 좌표에 대해서
            if not checked[y][x] and ice[y][x] > 0: # 얼음이 존재하고, 아직 체크하지 않은 좌표에 대해서
                checked[y][x] = True
                val, num = check_ice(N, ice, checked, y, x) # 연결된 얼음 덩어리의 총 얼음 개수와 얼음 덩어리의 크기 반환
                sum_val += val # 총 얼음 개수에 더해준다.
                max_val = max(num, max_val) # 최대 얼음 덩어리의 크기 갱신
    return sum_val, max_val # 총 얼음 개수, 최대 얼음 덩어리의 크기 반환
    
    
T = int(input("Testcase num: "))
for _ in range(T):
    print("\n\nInput")
    N, Q = map(int, input().split()) # 2^N크기의 격자, Q번의 파이어스톰
    ice = [] # 각 좌표에 지닌 얼음의 개수
    for _ in range(pow(2, N)):
        ice.append(list(map(int, input().split())))
    L = list(map(int, input().split())) # 파이어스톰 리스트
    print("\nOutput")
    total_ice, max_ice = simulation(N, Q, ice, L) # 시뮬레이션 진행
    print(total_ice) # 총 얼음 개수
    print(max_ice) # 최대 얼음 덩어리 크기

Testcase num: 6


Input
3 1
1 2 3 4 5 6 7 8
8 7 6 5 4 3 2 1
1 2 3 4 5 6 7 8
8 7 6 5 4 3 2 1
1 2 3 4 5 6 7 8
8 7 6 5 4 3 2 1
1 2 3 4 5 6 7 8
8 7 6 5 4 3 2 1
1

Output
284
64


Input
3 2
1 2 3 4 5 6 7 8
8 7 6 5 4 3 2 1
1 2 3 4 5 6 7 8
8 7 6 5 4 3 2 1
1 2 3 4 5 6 7 8
8 7 6 5 4 3 2 1
1 2 3 4 5 6 7 8
8 7 6 5 4 3 2 1
1 2

Output
280
64


Input
3 5
1 2 3 4 5 6 7 8
8 7 6 5 4 3 2 1
1 2 3 4 5 6 7 8
8 7 6 5 4 3 2 1
1 2 3 4 5 6 7 8
8 7 6 5 4 3 2 1
1 2 3 4 5 6 7 8
8 7 6 5 4 3 2 1
1 2 0 3 2

Output
268
64


Input
3 10
1 2 3 4 5 6 7 8
8 7 6 5 4 3 2 1
1 2 3 4 5 6 7 8
8 7 6 5 4 3 2 1
1 2 3 4 5 6 7 8
8 7 6 5 4 3 2 1
1 2 3 4 5 6 7 8
8 7 6 5 4 3 2 1
1 2 0 3 2 1 2 3 2 3

Output
248
62


Input
3 10
1 2 3 4 5 6 7 8
8 7 6 5 4 3 2 1
1 2 3 4 5 6 7 8
8 7 6 5 4 3 2 1
1 2 3 4 5 6 7 8
8 7 6 5 4 3 2 1
1 2 3 4 5 6 7 8
8 7 6 5 4 3 2 1
1 2 3 1 2 3 1 2 3 1

Output
246
60


Input
3 10
1 0 3 4 5 6 7 0
8 0 6 5 4 3 2 1
1 2 0 4 5 6 7 0
8 7 6 5 4 3 2 1
1 2 3 4 0 6 7 0
8 7 0 5 4 3 2 1
1 2 3 4 5 6 7 0
0 7 0 5 4 3 2 1
1 2 3 1 2 